In [1]:
from queue import PriorityQueue

In [29]:
#################################################################33

In [51]:
# Define the Manhattan distance heuristic function
def manhattan_distance(state):
    distance = 0
    for i in range(3):
        for j in range(3):
            tile = state[3*i+j]
            if tile != 0:
                x, y = (tile-1) // 3, (tile-1) % 3
                distance += abs(x-i) + abs(y-j)
    return distance


In [58]:
# алгоритм рекурсивного пошуку найкращого (RBFS) з обмеженням глибини
def rbfs_search(state, f_limit, g, num_operations):
    num_operations += 1
    # Перевіряємо, чи поточний стан є цільовим
    if state == goal_state:
        return [state], g, num_operations

    # Згенеруємо сусідні стани
    neighbors = []
    blank_tile_index = state.index(0)
    for move in [-1, 1, -3, 3]:
        neighbor_index = blank_tile_index + move
        if neighbor_index < 0 or neighbor_index >= 9:
            continue
        if move == -1 and neighbor_index == 2:
            continue
        if move == 1 and neighbor_index == 3:
            continue
        if move == -3 and neighbor_index == 6:
            continue
        if move == 3 and neighbor_index == 5:
            continue
        neighbor_state = state[:]
        neighbor_state[blank_tile_index], neighbor_state[neighbor_index] = neighbor_state[neighbor_index], neighbor_state[blank_tile_index]
        neighbors.append(neighbor_state)

    # Перевіряємо, чи немає поруч сусідів
    if not neighbors:
        return None, float('inf'), num_operations

    # Обчислимо f-рахунки сусідів
    for neighbor in neighbors:
        neighbor_f_score = max(manhattan_distance(neighbor) + g, f_limit)

    # Повторюємо виклик RBFS для сусіда з найнижчим f-score, доки не буде знайдено рішення
    while True:
        # сортуємо сусідів за f-score
        neighbors.sort(key=lambda neighbor: max(manhattan_distance(neighbor) + g, f_limit))

        # отримуємо кращого сусіда
        best_neighbor = neighbors[0]
        best_neighbor_f_score = max(manhattan_distance(best_neighbor) + g, f_limit)

        # Перевіримо, чи перевищує f-score найкращого сусіда f-limit
        if best_neighbor_f_score > f_limit:
            return None, best_neighbor_f_score, num_operations

        # Обчислити альтернативний f-score найкращого сусіда
        alternative_f_score = float('inf')
        if len(neighbors) > 1:
            alternative_f_score = max(manhattan_distance(neighbors[1]) + g, f_limit)

        # Рекурсивно викликаємо RBFS для найкращого сусіда з альтернативним f-score
        result, best_neighbor_f_score, num_operations = rbfs_search(best_neighbor, min(f_limit, alternative_f_score), g+1, num_operations)

        # Перевіряємо, чи знайдено рішення
        if result is not None:
            return [state] + result, best_neighbor_f_score, num_operations

In [61]:
%%time
#RBFS
goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
initial_state = [1, 0, 3, 4, 2, 5, 7, 8, 6]
f_limit = manhattan_distance(initial_state)
g = 0 # вартість шляху від початкового стану до поточного 
num_operations = 0
while True:
    solution, f_limit, num_operations = rbfs_search(initial_state, f_limit, g, num_operations)
    if result is not None:
        print('Solution found:', solution[-1])
        print('Depth of the solution:', f_limit)
        print('Count operations:', num_operations)
        print('Solution Path:', solution)
        break

Solution found: [1, 2, 3, 4, 5, 6, 7, 8, 0]
Depth of the solution: 3
Count operations: 4
Solution Path: [[1, 0, 3, 4, 2, 5, 7, 8, 6], [1, 2, 3, 4, 0, 5, 7, 8, 6], [1, 2, 3, 4, 5, 0, 7, 8, 6], [1, 2, 3, 4, 5, 6, 7, 8, 0]]
Wall time: 0 ns


In [54]:
#LDFS algorithm
def limited_dfs_search(state, limit):
    # Перевіряємо, чи поточний стан є цільовим
    if state == goal_state:
        return [state]

    # Check if the depth limit has been reached
    if limit == 0:
        return None

    # Згенеруємо сусідні стани
    blank_tile_index = state.index(0)
    for move in [-1, 1, -3, 3]:
        neighbor_index = blank_tile_index + move
        if neighbor_index < 0 or neighbor_index >= 9:
            continue
        if move == -1 and neighbor_index == 2:
            continue
        if move == 1 and neighbor_index == 3:
            continue
        if move == -3 and neighbor_index == 6:
            continue
        if move == 3 and neighbor_index == 5:
            continue
        neighbor_state = state[:]
        neighbor_state[blank_tile_index], neighbor_state[neighbor_index] = neighbor_state[neighbor_index], neighbor_state[blank_tile_index]
        # Рекурсивний пошук сусідніх станів зі зменшеною глибиною
        result = limited_dfs_search(neighbor_state, limit-1)
        if result is not None:
            return [state] + result
    global num_operations
    num_operations += 1

    # Return None якщо розв'язок не знайдено
    return None

# Визначаємо алгоритм ітеративного пошуку з поглибленням
def iterative_deepening_search(initial_state):
    # Ініціалізуємо межу глибини та кількість операцій
    limit = 0
    global num_operations
    num_operations = 0

    # Збільшуємо граничну глибину, доки не буде знайдено рішення
    while True:
        result = limited_dfs_search(initial_state, limit)
        if result is not None:
            return result
        limit += 1

In [64]:
%%time
#LDFS
goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
initial_state = [1, 0, 3, 4, 2, 5, 7, 8, 6]
solution = iterative_deepening_search(initial_state)
if solution is not None:
    print("Solution found:", solution[-1])
    print("Number of operations:", num_operations)
    print("Solution path: ", solution)
else:
    print("No solution found.")

Solution found: [1, 2, 3, 4, 5, 6, 7, 8, 0]
Number of operations: 11
Solution path:  [[1, 0, 3, 4, 2, 5, 7, 8, 6], [1, 2, 3, 4, 0, 5, 7, 8, 6], [1, 2, 3, 4, 5, 0, 7, 8, 6], [1, 2, 3, 4, 5, 6, 7, 8, 0]]
Wall time: 0 ns
